In [1]:
import random
import pandas as pd
import glob
import os
import numpy as np
import pickle as pkl
import time
#from fastparquet import write
#import fastparquet
#import dask.dataframe as dd
#import snappy
from pandas.api.types import union_categoricals
#import multiprocessing as mp
#from multiprocessing import Pool
#from pandarallel import pandarallel
import re

dask.config.config

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=50,threads_per_worker=1,scheduler_port=0,memory_limit=0)
client = Client(cluster)

cluster.adapt(minimum=10, maximum=80)

In [20]:
pkl_files=glob.glob('/data/dharp/compounds/datasets/compounds/*pkl')
random.shuffle(pkl_files)
len(pkl_files)

115

In [21]:
div_lsts=np.array_split(pkl_files, 10)

In [22]:
len(div_lsts[0])

12

In [23]:
keep_string=r"^.+_(PROPN|NOUN|ADJ|VERB||NUM|ADV)$"

In [24]:
temp_df=pd.read_pickle(div_lsts[5][1])

In [25]:
temp_df=temp_df.tail(1_000_000).copy()

In [26]:
temp_df.sample(30)

,modifier,head,num_comp,context,year,comp_ner_sent,count
90563680,year_NOUN,storm_NOUN,True,and_CCONJ,2010,,6
90770896,zat_PROPN,arras_PROPN,True,upon_SCONJ,2011,PERSON,2
90963799,°_NOUN,c_NOUN,True,darkness_NOUN,2003,,1
90260334,year_NOUN,course_NOUN,True,study_NOUN,1977,,3
90688506,ynyl_NOUN,group_NOUN,True,2_NUM,1966,,10
90926403,°_NOUN,c_NOUN,False,-12_NUM,1989,,1
90596180,year_NOUN,term_NOUN,True,enlistment_NOUN,2001,,4
90699752,yon_PROPN,streak_PROPN,True,see_VERB,1964,,2
90309786,year_NOUN,extension_NOUN,True,but_CCONJ,2019,DATE,1
90543573,year_NOUN,sentence_NOUN,True,in_ADP,1934,,6


In [8]:
temp_df.comp_ner_sent.value_counts(normalize=True)

                    4.061540e-01
PERSON              1.332132e-01
ORG                 1.157497e-01
PERSON PERSON       7.894330e-02
ORG ORG             7.570075e-02
                        ...     
DATE ORG DATE       1.354090e-08
ORDINAL LOC LOC     1.354090e-08
FAC FAC GPE FAC     1.354090e-08
GPE GPE DATE ORG    6.770452e-09
PRODUCT FAC FAC     6.770452e-09
Name: comp_ner_sent, Length: 803, dtype: float64

In [10]:
temp_df.loc[temp_df.comp_ner_sent=='PERSON']

,modifier,head,num_comp,context,year,comp_ner_sent,count
38991,%_NOUN,corporation_NOUN,True,21_NUM,1997,PERSON,10
38992,%_NOUN,corporation_NOUN,True,21_NUM,1998,PERSON,20
38993,%_NOUN,corporation_NOUN,True,21_NUM,1999,PERSON,14
38994,%_NOUN,corporation_NOUN,True,21_NUM,2000,PERSON,1
38995,%_NOUN,corporation_NOUN,True,foundation_PROPN,1997,PERSON,10
...,...,...,...,...,...,...,...
443101786,_PROPN,_NOUN,True,_PROPN,2011,PERSON,4
443101787,_PROPN,_NOUN,True,_PROPN,2012,PERSON,1
443101788,_PROPN,_NOUN,True,_PROPN,2013,PERSON,1
443101789,_PROPN,_NOUN,True,_PROPN,2014,PERSON,1


In [9]:
def mem_reducer(pkl_file):
    print(f'\nStarted with file {pkl_file}\n')
    cur_time=time.time()
    
    df=pd.read_pickle(pkl_file)
    context_df=pd.DataFrame(df['context'].unique())
    context_df=context_df.loc[~context_df[0].str.contains('@@@')]
    context_words=context_df.loc[context_df[0].str.match(keep_string)][0].to_list()
    df=df.loc[df.context.isin(context_words)]
    df.context=df.context.str.replace(r'.+_NUM','NUM',regex=True)
    df["year"] = pd.to_numeric(df["year"], downcast="unsigned")
    df["comp_ner_sent"] = df["comp_ner_sent"].astype("category")
    df=df.groupby(['modifier','head','num_comp','context','year','comp_ner_sent'],observed=True)['count'].sum()
    df=df.to_frame().reset_index()    
    print(f'Done with file {pkl_file} in {round(time.time()-cur_time)} secs')
    return df

In [10]:
for cur_list in div_lsts[:1]:
    n_proc = len(cur_list)

    pool = Pool(n_proc)
    results=pool.map_async(mem_reducer,cur_list) 
    pool.close()
    pool.join()
    


Started with file /data/dharp/compounds/datasets/phrases/df_23.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_13.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_46.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_5.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_98.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_49.pkl


Started with file /data/dharp/compounds/datasets/phrases/df_75.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_14.pkl


Started with file /data/dharp/compounds/datasets/phrases/df_21.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_96.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_95.pkl

Started with file /data/dharp/compounds/datasets/phrases/df_107.pkl










Done with file /data/dharp/compounds/datasets/phrases/df_13.pkl in 400 secs
Done with file /data/dharp/compounds/datasets/phrases/df_5.pkl in 468 secs
Done with file /data/

In [12]:
dfs=results.get()

In [13]:
union_cat_ner = union_categoricals([df.comp_ner_sent for df in dfs])

for df in dfs:
    df.comp_ner_sent=pd.Categorical(df.comp_ner_sent,categories=union_cat_ner.categories)

In [14]:
df=pd.concat(dfs,ignore_index=True,sort=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794893936 entries, 0 to 794893935
Data columns (total 7 columns):
 #   Column         Dtype   
---  ------         -----   
 0   comp_ner_sent  category
 1   context        object  
 2   count          int64   
 3   head           object  
 4   modifier       object  
 5   num_comp       bool    
 6   year           uint64  
dtypes: bool(1), category(1), int64(1), object(3), uint64(1)
memory usage: 31.8+ GB


In [23]:
import gc
del ddf
gc.collect()

4508

In [25]:
df=df.groupby(['modifier','head','num_comp','context','year','comp_ner_sent'],observed=True,sort=False)['count'].sum()
df=df.to_frame().reset_index() 

In [29]:
df

,modifier,head,num_comp,context,year,comp_ner_sent,count
0,#_NOUN,#_NOUN,False,#_NOUN,1802,,1
1,#_NOUN,#_NOUN,False,#_NOUN,1810,,4
2,#_NOUN,#_NOUN,False,#_NOUN,1812,,2
3,#_NOUN,#_NOUN,False,#_NOUN,1815,,2
4,#_NOUN,#_NOUN,False,#_NOUN,1819,,6
...,...,...,...,...,...,...,...
710373849,_PROPN,_NOUN,True,_PROPN,2011,,1
710373850,_PROPN,_NOUN,True,_PROPN,2012,,3
710373851,_PROPN,_NOUN,True,_PROPN,2013,,1
710373852,_PROPN,_NOUN,True,_PROPN,2014,,2


In [28]:
710373853/794893935

0.8936712455857397

In [3]:
pkl_df=pd.DataFrame(all_pkl_files,pkl_file_sizes)
pkl_df.reset_index(inplace=True)
pkl_df.columns=['fsize','fname']
pkl_df['fsize_perc']=pkl_df.fsize/pkl_df.fsize.sum()*100
pkl_df.sort_values(by=['fsize'],ascending=False,inplace=True,ignore_index=True)
pkl_df.fsize/=1024*1024
pkl_df

,fsize,fname,fsize_perc
0,14384.012010,/data/dharp/compounds/datasets/phrases/df_112.pkl,3.406725
1,14207.451440,/data/dharp/compounds/datasets/phrases/df_111.pkl,3.364908
2,13127.603033,/data/dharp/compounds/datasets/phrases/df_114.pkl,3.109155
3,12762.725824,/data/dharp/compounds/datasets/phrases/df_113.pkl,3.022737
4,11979.878458,/data/dharp/compounds/datasets/phrases/df_110.pkl,2.837327
...,...,...,...
75,1842.023335,/data/dharp/compounds/datasets/phrases/df_2.pkl,0.436267
76,1648.576798,/data/dharp/compounds/datasets/phrases/df_7.pkl,0.390451
77,1644.348684,/data/dharp/compounds/datasets/phrases/df_9.pkl,0.389449
78,1528.614932,/data/dharp/compounds/datasets/phrases/df_8.pkl,0.362039


In [4]:
maxvalue = 40_000

lastvalue = 0
newcum = []
labels=[]
cur_label=1
for row in pkl_df.itertuples():
    thisvalue =  row.fsize + lastvalue
    if thisvalue > maxvalue:
        thisvalue = 0
        cur_label+=1
    newcum.append(thisvalue)
    labels.append(cur_label)
    lastvalue = thisvalue
pkl_df['fcat']=labels
pkl_df

,fsize,fname,fsize_perc,fcat
0,14384.012010,/data/dharp/compounds/datasets/phrases/df_112.pkl,3.406725,1
1,14207.451440,/data/dharp/compounds/datasets/phrases/df_111.pkl,3.364908,1
2,13127.603033,/data/dharp/compounds/datasets/phrases/df_114.pkl,3.109155,2
3,12762.725824,/data/dharp/compounds/datasets/phrases/df_113.pkl,3.022737,2
4,11979.878458,/data/dharp/compounds/datasets/phrases/df_110.pkl,2.837327,2
...,...,...,...,...
75,1842.023335,/data/dharp/compounds/datasets/phrases/df_2.pkl,0.436267,10
76,1648.576798,/data/dharp/compounds/datasets/phrases/df_7.pkl,0.390451,10
77,1644.348684,/data/dharp/compounds/datasets/phrases/df_9.pkl,0.389449,10
78,1528.614932,/data/dharp/compounds/datasets/phrases/df_8.pkl,0.362039,10


In [5]:
pkl_df.groupby('fcat')['fsize_perc'].sum().to_list()

[6.771632219533384,
 11.78735019750231,
 10.825373722876131,
 10.190599109791647,
 10.563520324653819,
 10.157906408579452,
 10.325476443269524,
 10.090011386733876,
 10.224589719273242,
 9.063540467786614]

In [74]:
pkl_df.query('1 <= fcat <= 40').to_csv('/data/dharp/compounds/datasets/drogium_fcat.txt',sep="\t",index=False)

In [75]:
pkl_df.query('41 <= fcat <= 80').to_csv('/data/dharp/compounds/datasets/mithril_fcat.txt',sep="\t",index=False)

In [76]:
pkl_df.query('81 <= fcat <= 115').to_csv('/data/dharp/compounds/datasets/rivium_fcat.txt',sep="\t",index=False)

results[0][['modifier','head']].value_counts().sample(50)

def write_to_hdf(data_bin):
    if data_bin.shape[0]!=0:
        df_list=[]
        print(data_bin.shape[0])
        for row in data_bin.itertuples():
            #print(row.fname)
            tmp_pkl=pd.read_pickle(row.fname)
            tmp_pkl.reset_index(inplace=True)
            df_list.append(tmp_pkl)
        tmp_df=pd.concat(df_list,ignore_index=True,sort=False)
        print('Done concatenating')
        tmp_df.groupby(['fivegram_pos','year'])['count'].sum().to_frame().reset_index(inplace=True)
        print('Done grouping')
        tmp_df.to_hdf(path+'/entire_df_'+str(row.fcat)+'.h5',format='table', key='df',complib='zlib', complevel=5)

In [50]:
def write_to_parquet(data_bin):
    print(data_bin.iloc[0].fcat)

    cur_time=time.time()
    df_list=[]
    print(data_bin.shape[0])
    for row in data_bin.itertuples():
        #print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        cur_df.reset_index(inplace=True)
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')
    
    ddf = dd.from_pandas(concat_df, npartitions=100)
    
    ddf=ddf.groupby(['lemma_pos','year','comp_class'])['count'].sum()
    print('Done grouping')
    ddf=ddf.to_frame().reset_index().compute()
    
    print(ddf.shape[0])
    after_shape=ddf.shape[0]
    
    ddf.to_parquet(
    path=f'{save_path}/df_{row.fcat}.parq', 
    engine='fastparquet',
    compression='snappy',
    row_group_offsets=25_000_000)
    print(f"Finished df {row.fcat} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f'Time taken {time.time()-cur_time} secs')

In [51]:
def reader(fname):
    
    cur_df=pd.read_pickle(fname)
    
    if cur_df.shape[1]!=7:
        print(f'{fname}')
        display(cur_df.head())
  

In [7]:
def test(data_bin):
    print(data_bin.iloc[0].fcat)

    cur_time=time.time()
    df_list=[]

    for row in data_bin.itertuples():
        #print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        cur_df.reset_index(inplace=True)
        cur_df=cur_df.head(100_000)
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')
    
    ddf = dd.from_pandas(concat_df, npartitions=100)
    
    ddf=ddf.groupby(['lemma_pos','year','comp_class'])['count'].sum()
    print('Done grouping')
    ddf=ddf.to_frame().reset_index().compute()
    
    print(ddf.shape[0])
    after_shape=ddf.shape[0]
    print(ddf.info())
    #ddf.to_parquet(
    #path=f'{save_path}/df_{row.fcat}.parq', 
    #engine='fastparquet',
    #compression='snappy',
    #row_group_offsets=10_000_000)
    print(f"Finished df {row.fcat} ; Before : {total_df_shape}, After : {after_shape} Change in percentage : {(total_df_shape-after_shape)/total_df_shape*100:0.2f}%")
    print(f'Time taken {time.time()-cur_time} secs')   

In [27]:
save_path='/data/dharp/compounds/datasets/entire_df_v3/'


In [54]:
test_df=pkl_df.loc[pkl_df.fcat==92]

,fsize,fname,fsize_perc,fcat
5709,108.528206,/data/dharp/compounds/datasets/googleV3/1196.pkl,2.285763e-03,92
5710,108.415882,/data/dharp/compounds/datasets/googleV3/14522.pkl,2.283397e-03,92
5711,108.209579,/data/dharp/compounds/datasets/googleV3/11475.pkl,2.279052e-03,92
5712,108.091488,/data/dharp/compounds/datasets/googleV3/2815.pkl,2.276565e-03,92
5713,107.772347,/data/dharp/compounds/datasets/googleV3/13463.pkl,2.269843e-03,92
...,...,...,...,...
6525,0.019082,/data/dharp/compounds/datasets/googleV3/1631.pkl,4.018963e-07,92
6526,0.017426,/data/dharp/compounds/datasets/googleV3/13021.pkl,3.670274e-07,92
6527,0.011068,/data/dharp/compounds/datasets/googleV3/16262.pkl,2.331155e-07,92
6528,0.010059,/data/dharp/compounds/datasets/googleV3/2210.pkl,2.118648e-07,92


In [ ]:
    cur_time=time.time()
    df_list=[]
    print(test_df.shape[0])
    for row in test_df.itertuples():
        print(row.fname)
        cur_df=pd.read_pickle(row.fname)
        print(cur_df.shape)
        #print(cur_df.head())
        #cur_df.reset_index(inplace=True)
        #print(cur_df.head())
        df_list.append(cur_df)
    concat_df=pd.concat(df_list,ignore_index=True,sort=False)
    print(concat_df.shape[0])
    total_df_shape=concat_df.shape[0]
    print('Done concatenating')

In [60]:
concat_df.comp_class.value_counts(normalize=True)

0    0.834707
2    0.060470
4    0.045259
5    0.032317
3    0.015007
1    0.012240
Name: comp_class, dtype: float64

In [63]:
    ddf = dd.from_pandas(concat_df, npartitions=120)
    
    ddf=ddf.groupby(['lemma_pos','pos_sent','year','comp_class','num_comp','comp_ner_sent'])['count'].sum()
    print('Done grouping')
    ddf=ddf.to_frame().reset_index().compute()
    
    print(ddf.shape[0])
    after_shape=ddf.shape[0]

Done grouping
1063450343


In [29]:
import fastparquet

In [34]:
cur_parq=fastparquet.ParquetFile('/data/dharp/compounds/datasets/entire_df_v3/df_1.parq')

In [36]:
df = cur_parq.to_pandas()

RuntimeError: Decompression 'SNAPPY' not available.  Options: ['BROTLI', 'GZIP', 'UNCOMPRESSED']